# Nencioli

In [ ]:
import pickle
with open('/srv/scratch/z5297792/Sample/U.pkl', 'rb') as file:
    U = pickle.load(file)
with open('/srv/scratch/z5297792/Sample/V.pkl', 'rb') as file:
    V = pickle.load(file)

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import time


In [ ]:
def nenc_crit_1_2(u, v, a):
    rows_v, cols_v = v.shape
    rows_u, cols_u = u.shape
    if rows_v != rows_u or cols_v != cols_u:
        raise ValueError("The shapes of 'v' and 'u' must be the same.")

    criteria_mask_1 = np.zeros_like(v, dtype=bool)
    criteria_mask_2 = np.zeros_like(u, dtype=bool)

    # First function nenc_crit_1
    for j in range(a, cols_v - a):
        wrk = v[:, j]
        s = np.sign(wrk)
        indx = np.where(np.diff(s) != 0)[0]
        for ii in indx:
            if ii - a >= 0 and ii + a < rows_v:  # Check boundaries
                if v[ii - a, j] * v[ii + a, j] < 0:
                    if abs(v[ii - a, j]) > abs(v[ii, j]) and abs(v[ii + a, j]) > abs(v[ii, j]):
                        criteria_mask_1[ii, j] = True

    # Second function nenc_crit_2, only checking where criteria_mask_1 is True
    for i in range(a, rows_u - a):
        wrk = u[i, :]
        s = np.sign(wrk)
        indx = np.where(np.diff(s) != 0)[0]
        for jj in indx:
            if criteria_mask_1[i, jj]:  # Only check where nenc_crit_1 is True
                if jj - a >= 0 and jj + a < cols_u:  # Check boundaries
                    if u[i, jj - a] * u[i, jj + a] < 0:
                        if abs(u[i, jj - a]) > abs(u[i, jj]) and abs(u[i, jj + a]) > abs(u[i, jj]):
                            criteria_mask_2[i, jj] = True

    return criteria_mask_2

def nenc_crit_3(u, v, b):
    velocity_magnitude = np.sqrt(u**2 + v**2)
    rows, cols = velocity_magnitude.shape
    local_minimum_mask = np.zeros_like(velocity_magnitude, dtype=bool)
    for i in range(b, rows - b):  
        for j in range(b, cols - b):  
            current_magnitude = velocity_magnitude[i, j]
            neighborhood = velocity_magnitude[i-b:i+b+1, j-b:j+b+1]
            if current_magnitude <= np.min(neighborhood):
                local_minimum_mask[i, j] = True
                # Set the 8 neighboring points to True, with boundary checks
                for di in [-1, 0, 1]:
                    for dj in [-1, 0, 1]:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < rows and 0 <= nj < cols:  # Ensure neighbors are within bounds
                            local_minimum_mask[ni, nj] = True    
    return local_minimum_mask

def nenc_crit_4(u, v, a):

    # Assuming u and v are your 2D matrices of velocities
    rows, cols = u.shape

    # Create a mask to store points that meet the rotational consistency criterion
    rotation_mask = np.zeros_like(u, dtype=bool)

    def is_cyclic_rotation(lst):
        # Define allowed transitions for both patterns
        valid_pairs_1 = {(1, 2), (2, 3), (3, 4), (4, 1),  # Pattern 1: Ascending sequence
                         (1, 1), (2, 2), (3, 3), (4, 4)}  
        valid_pairs_2 = {(2, 1), (3, 2), (4, 3), (1, 4),  # Pattern 2: Descending sequence
                         (1, 1), (2, 2), (3, 3), (4, 4)}  
        
        # Helper function to check if the list satisfies a given set of valid pairs
        def check_valid(lst, valid_pairs):
            for i in range(1, len(lst)):
                if (lst[i-1], lst[i]) not in valid_pairs:
                    return False
            return True
    
        # Check both conditions
        check_1 = check_valid(lst, valid_pairs_1)
        check_2 = check_valid(lst, valid_pairs_2)
    
        # Return True if exactly one condition is satisfied
        return (check_1 or check_2) and not (check_1 and check_2)
    
    # Function to determine the quadrant of an angle
    def quadrant_check(u_bound, v_bound):
        u_bound, v_bound = np.array(u_bound), np.array(v_bound)
        quadrants = np.zeros(u_bound.shape, dtype=int)
        quadrants[(u_bound >= 0) & (v_bound >= 0)] = 1
        quadrants[(u_bound < 0) & (v_bound >= 0)] = 2
        quadrants[(u_bound < 0) & (v_bound < 0)] = 3
        quadrants[(u_bound >= 0) & (v_bound < 0)] = 4

        if np.any(quadrants==1) and np.any(quadrants==2) and np.any(quadrants==3) and np.any(quadrants==4):
            if is_cyclic_rotation(quadrants):
                return True
        
        return False

    # Function to check if two quadrants are adjacent
    def adjacent_quadrants(q1, q2):
        return abs(q1 - q2) == 1 or abs(q1 - q2) == 3  # Wrap-around (1 and 4 are adjacent)
        
    # Loop through each point (avoid the edges by 'a' grid points)
    for i in range(a, rows - a):
        for j in range(a, cols - a):
  
            u_bound = []
            u_bound.extend(u[i-a:i+a+1,j+a])
            u_bound.extend(np.flip(u[i+a,j-a:j+a+1]))
            u_bound.extend(np.flip(u[i-a:i+a+1,j-a]))
            u_bound.extend(u[i-a,j-a:j+a+1])

            v_bound = []
            v_bound.extend(v[i-a:i+a+1,j+a])
            v_bound.extend(np.flip(v[i+a,j-a:j+a+1]))
            v_bound.extend(np.flip(v[i-a:i+a+1,j-a]))
            v_bound.extend(v[i-a,j-a:j+a+1])
            
            if quadrant_check(u_bound, v_bound):
                rotation_mask[i, j] = True

    return rotation_mask


In [ ]:
depth_lvl = 15
u, v = U[:,:,depth_lvl], V[:,:,depth_lvl]
a, b = 4, 3
combined = np.logical_and.reduce((nenc_crit_1_2(u, v, a), nenc_crit_3(u, v, b), nenc_crit_4(u, v, a)))
ic, jc = np.where(combined)

x, y = np.arange(0, u.shape[0]), np.arange(0, u.shape[1])

spacing = 5
plt.quiver(x[::spacing], y[::spacing], U[::spacing,::spacing,depth_lvl].T,V[::spacing,::spacing,depth_lvl].T)
plt.scatter(ic, jc)

fig, axs = plt.subplots(1, 3, figsize=(14,3))

axs[0].spy(nenc_crit_1_2(u, v, a))
axs[0].set_title('Crit. 1 and 2')
axs[1].spy(nenc_crit_3(u, v, b))
axs[1].set_title('Crit. 3')
axs[2].spy(nenc_crit_4(u, v, a))
axs[2].set_title('Crit. 4')

In [ ]:
def nencioli(u, v, a, b, cyclonicity):
    rows_v, cols_v = v.shape
    rows_u, cols_u = u.shape
    if rows_v != rows_u or cols_v != cols_u:
        raise ValueError("The shapes of 'v' and 'u' must be the same.")
    rows, cols = rows_v, cols_v
    
    borders = np.max(a, b) + 1

    vel = np.sqrt(u**2 + v**2)

    eddy_uv = [0, 0]

    eddy_c = [0, 0]

    eddy = [0, 0, 0]

    bound = vel.shape

    for i in range(borders, rows - borders + 1):
        
        # 1st constraint - v comoponent
        wrk = v[i, :]
        s = np.sign(wrk)
        indx = np.where((np.diff(s) != 0) & (~np.isnan(np.diff(s))))[0]
        indx = indx[(indx >= borders) & (indx <= cols - borders)]

        for ii in range(len(indx)):
            
            if wrk[indx[ii]] >= 0: # AE
                if (wrk[indx[ii] - a] > wrk[indx[ii]]) & (wrk[indx[ii] + 1 + a] < wrk[indx[ii] + 1]):
                    var = -1
                else:
                    var = 0
            elif wrk[indx[ii]] < 0: # CE
                if (wrk[indx[ii] - a] < wrk[indx[ii]]) & (wrk[indx[ii] + 1 + a] > wrk[indx[ii] + 1]):
                    var = 1
                else:
                    var = 0
       
            # 2nd constraint - u comoponent
            if var == -1: # AE
                if ((u[i - a, indx[ii]] <= 0) & (u[i - a, indx[ii]] <= u[i - 1, indx[ii]]) & \
                     (u[i + a, indx[ii]] >= 0) & (u[i + a, indx[ii]] >= u[i + 1, indx[ii]])) | \
                    ((u[i - a, indx[ii] + 1] <= 0) & (u[i - a, indx[ii] + 1] <= u[i - 1, indx[ii] + 1]) & \
                     (u[i + a, indx[ii] + 1] >= 0) & (u[i + a, indx[ii] + 1] >= u[i + 1, indx[ii] + 1])):
                    var = -1
                    eddy_uv = np.vstack([
                        eddy_uv,
                        [i, indx[ii]],
                        [i, indx[ii] + 1]
                    ])
                else:
                    var = 0
                
            elif var == 1: # CE
                if ((u[i - a, indx[ii]] >= 0) & (u[i - a, indx[ii]] >= u[i - 1, indx[ii]]) & \
                     (u[i + a, indx[ii]] <= 0) & (u[i + a, indx[ii]] <= u[i + 1, indx[ii]])) | \
                    ((u[i - a, indx[ii] + 1] >= 0) & (u[i - a, indx[ii] + 1] >= u[i - 1, indx[ii] + 1]) & \
                     (u[i + a, indx[ii] + 1] <= 0) & (u[i + a, indx[ii] + 1] <= u[i + 1, indx[ii] + 1])):
                    var = 1
                    eddy_uv = np.vstack([
                        eddy_uv,
                        [i, indx[ii]],
                        [i, indx[ii] + 1]
                    ])
                else:
                    var = 0
            
            # 3rd constraint - velocity minimum
            if var != 0:
                srch = vel[i - b:i + b + 1, indx[ii] - b:indx[ii] + b + 1]
                si = np.arange(i - b, i + b + 1)
                sj = np.arange(indx[ii] - b, indx[ii] + b + 1)

                min_indices = np.argwhere(srch == np.min(srch))
                X, Y = min_indices[:, 0], min_indices[:, 1]

                x_start = max((i - b) + (X - 1) - b, 0)
                x_end = min((i - b) + (X - 1) + b, bound[0])
                y_start = max((indx[ii] - b) + (Y - 1) - b, 0)
                y_end = min((indx[ii] - b) + (Y - 1) + b, bound[1])

                srch2 = vel[x_start:x_end + 1, y_start:y_end + 1]

                if not (np.min(srch2) == np.min(srch)):
                    var = 0
                else:
                    eddy_c = np.vstack([eddy_c, [si[X[0]], sj[Y[0]]]])

            # 4th constraint - vector rotation
            d = a - 1
            if var != 0:
                i1, i2 = np.where((lat == slat[X[0], Y[0]]) & (lon == slon[X[0], Y[0]]))
                
                u_small = u[max(i1 - d, 0):min(i1 + d + 1, bound[0]), 
                            max(i2 - d, 0):min(i2 + d + 1, bound[1])]
                
                v_small = v[max(i1 - d, 0):min(i1 + d + 1, bound[0]), 
                            max(i2 - d, 0):min(i2 + d + 1, bound[1])]
                
                if not np.any(np.isnan(u_small)):

                    u_bound = np.concatenate([u_small[0, :], u_small[1:, -1], u_small[-1, -2::-1], u_small[-2::-1, 0]])
                    v_bound = np.concatenate([v_small[0, :], v_small[1:, -1], v_small[-1, -2::-1], v_small[-2::-1, 0]])
                    
                    quadrants = np.zeros_like(u_bound, dtype=int)
                    
                    quadrants[(u_bound >= 0) & (v_bound >= 0)] = 1
                    quadrants[(u_bound < 0) & (v_bound >= 0)] = 2
                    quadrants[(u_bound < 0) & (v_bound < 0)] = 3
                    quadrants[(u_bound >= 0) & (v_bound < 0)] = 4

                    spin = np.where(quadrants == 4)[0]

                    if spin.size > 0 and spin.shape[0] != quadrants.shape[0]:
                        if spin[0] == 1:
                            spin = np.where(quadrants != 4)[0]
                            spin = spin[0] - 1
                        
                        quadrants[spin[-1]+1:] = quadrants[spin[-1]+1:] + 4

                        if not np.any(np.diff(quadrants) > 1) and not np.any(np.diff(quadrants) < 0):
                            eddy = np.vstack([eddy, [si[X[0]], sj[Y[0]], var]])

    def process_eddy(eddy):
        # Sort by longitude (column 1) and latitude (column 2)
        eddy = eddy[np.argsort(eddy[:, 1])]  # Sort by lat (column 2)
        eddy = eddy[np.argsort(eddy[:, 0])]  # Then sort by lon (column 1)
        
        # Keep only one value per eddy center
        diff_lon = np.diff(eddy[:, 0]) != 0
        diff_lat = np.diff(eddy[:, 1]) != 0
        good = np.where(diff_lon | diff_lat)[0]
        eddy = eddy[np.r_[0, good + 1]]  # Keep first and valid indices
        
        # Remove rows where all values are zero
        zero_eddy = np.where((eddy[:, 0] == 0) & (eddy[:, 1] == 0) & (eddy[:, 2] == 0))[0]
        eddy = np.delete(eddy, zero_eddy, axis=0)
        
        return eddy
    
    # Process each eddy array (eddy, eddy_uv, eddy_c)
    eddy = process_eddy(eddy)
    eddy_uv = process_eddy(eddy_uv)
    eddy_c = process_eddy(eddy_c)



In [ ]:
1+1